# ManaTTS Crawling
This notebook includes the scripts used to crawl the ManaTTS dataset raw audio and text files from the [Nasl-e-Mana](https://naslemana.com/) magazine. Notice that some initial issues of this magazine are available on the [IBNGO](ibngo.ir) website and cannot be crawled. These issues were downloaded manually in the ManaTTS dataset.

# Crawl audio-transcription pairs

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import logging
from google.colab import drive

In [ ]:
# Configure logging for link not on naslemana.com
logging.basicConfig(filename='naslemana_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Define the base URL and the directory to save files
base_url = "https://naslemana.com/"

save_dir = "raw-data/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# Function to download a file
def download_file(url, save_path):
    if not url.startswith(base_url):
        logging.info(f"Link not starting with naslemana.com: {url}")
        return
    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as out_file:
        out_file.write(response.content)

In [ ]:
# Function to extract text from a page
def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('span', class_='post-title', itemprop='headline').text
    subtitle = soup.find('h2', class_='post-subtitle')
    if subtitle:
        subtitle = subtitle.text
    else:
        subtitle = ""
    body_text = " ".join([p.text for p in soup.find_all('div', class_='entry-content clearfix single-post-content')])
    return title, subtitle, body_text

In [ ]:
# Initialize the metadata DataFrame
metadata_df = pd.DataFrame(columns=["magazine_name", "magazine_url", "subject", "audio_url", "text_url", "file_name"])

In [ ]:
# Crawl the pages
for page_num in range(1, 6):
    page_url = f"{base_url}page/{page_num}/"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find candidate pages
    candidate_pages = soup.find_all('a', class_='read-more')
    for candidate in candidate_pages:
        candidate_url = candidate['href']
        candidate_response = requests.get(candidate_url)
        candidate_soup = BeautifulSoup(candidate_response.text, 'html.parser')

        # Find elements with "(صوت)" and a link to an .mp3 file
        audio_elements = candidate_soup.find_all('a', href=lambda href: href and href.endswith('.mp3'), string=lambda string: string and '(صوت)' in string)

        if audio_elements:    # The candidate page is for a magazine
          print(f"Crawling {candidate_url}")

        for audio_element in audio_elements:
            audio_url = audio_element['href']
            audio_text = audio_element.text.replace('(صوت)', '').strip()

            # Look for a corresponding element with "(متن)"
            text_element = candidate_soup.find('a', string=lambda string: string and '(متن)' in string and audio_text in string)
            if text_element:
                text_url = text_element['href']

                # Extract text from the text page
                title, subtitle, body_text = extract_text(text_url)

                # Prepare the subject by removing specific words
                subject = title.replace('(متن)', '').replace('(صوت)', '')

                print(f"Downloading the pair: {subject}")

                # Download the audio file
                file_name = f"{save_dir}/{len(metadata_df) + 164}.mp3"
                download_file(audio_url, file_name)

                # Save the text to a file
                text_file_name = f"{save_dir}/{len(metadata_df) + 164}.txt"
                with open(text_file_name, 'w', encoding='utf-8') as f:
                    f.write(f"{title}\n{subtitle}\n{body_text}")

                # Update the metadata
                metadata_df = pd.concat([metadata_df, pd.DataFrame([{
                    "magazine_name": candidate_soup.find('span', class_='post-title', itemprop='headline').text,
                    "magazine_url": candidate_url,
                    "subject": subject,
                    "audio_url": audio_url,
                    "text_url": text_url,
                    "file_name": f"{len(metadata_df) + 164}"
                }])], ignore_index=True)

                # Save the metadata to a CSV file after each pair
                metadata_df.to_csv(f"{save_dir}/metadata.csv", index=False)

Crawling https://naslemana.com/2024/02/19/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9%84-%d9%85%d8%a7%d9%86%d8%a7-%d8%b3%d8%a7%d9%84-%d8%b3%d9%88%d9%85-%d8%b4%d9%85%d8%a7%d8%b1%d9%87-%db%b2%db%b6-%d8%a8%d9%87/
Crawling https://naslemana.com/2024/01/20/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9%84-%d9%85%d8%a7%d9%86%d8%a7-%d8%b3%d8%a7%d9%84-%d8%b3%d9%88%d9%85-%d8%b4%d9%85%d8%a7%d8%b1%d9%87-%db%b2%db%b5-%d8%af%db%8c/
Crawling https://naslemana.com/2023/12/21/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9%84-%d9%85%d8%a7%d9%86%d8%a7-%d8%b3%d8%a7%d9%84-%d8%af%d9%88%d9%85-%d8%b4%d9%85%d8%a7%d8%b1%d9%87-%db%b2%db%b4-%d8%a2%d8%b0/
Crawling https://naslemana.com/2023/11/21/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9%84-%d9%85%d8%a7%d9%86%d8%a7-%d8%b3%d8%a7%d9%84-%d8%af%d9%88%d9%85-%d8%b4%d9%85%d8%a7%d8%b1%d9%87-%db%b2%db%b3-%d8%a2%d8%a8/
Crawling https://naslemana.com/2023/10/22/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9

# Add new magazine
In case you want to crawl a single magazine, paste its url to the `candidate_url` variable and set the `last_available_file` variable to the gretest index of the previously crawled raw files. Then run the following code.

In [ ]:
# Initialize the metadata DataFrame
metadata_df = pd.DataFrame(columns=["magazine_name", "magazine_url", "subject", "audio_url", "text_url", "file_name"])

In [ ]:
last_available_file = 552
candidate_url = 'https://naslemana.com/2024/05/20/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9%84-%d9%85%d8%a7%d9%86%d8%a7-%d8%b3%d8%a7%d9%84-%d8%b3%d9%88%d9%85-%d8%b4%d9%85%d8%a7%d8%b1%d9%87-%db%b2%db%b9-%d8%a7%d8%b1/'

In [ ]:
candidate_response = requests.get(candidate_url)
candidate_soup = BeautifulSoup(candidate_response.text, 'html.parser')

# Find elements with "(صوت)" and a link to an .mp3 file
audio_elements = candidate_soup.find_all('a', href=lambda href: href and href.endswith('.mp3'), string=lambda string: string and '(صوت)' in string)


if audio_elements:    # The candidate page is for a magazine
  print(f"Crawling {candidate_url}")

for audio_element in audio_elements:
    audio_url = audio_element['href']
    audio_text = audio_element.text.replace('(صوت)', '').strip()

    # Look for a corresponding element with "(متن)"
    text_element = candidate_soup.find('a', string=lambda string: string and '(متن)' in string and audio_text in string)
    if text_element:
        text_url = text_element['href']

        # Extract text from the text page
        title, subtitle, body_text = extract_text(text_url)

        # Prepare the subject by removing specific words
        subject = title.replace('(متن)', '').replace('(صوت)', '')

        print(f"Downloading the pair: {subject}")

        # Download the audio file
        file_name = f"{save_dir}/{len(metadata_df) + last_available_file}.mp3"
        download_file(audio_url, file_name)

        # Save the text to a file
        text_file_name = f"{save_dir}/{len(metadata_df) + last_available_file}.txt"
        with open(text_file_name, 'w', encoding='utf-8') as f:
            f.write(f"{title}\n{subtitle}\n{body_text}")

        # Update the metadata
        metadata_df = pd.concat([metadata_df, pd.DataFrame([{
            "magazine_name": candidate_soup.find('span', class_='post-title', itemprop='headline').text,
            "magazine_url": candidate_url,
            "subject": subject,
            "audio_url": audio_url,
            "text_url": text_url,
            "file_name": f"{len(metadata_df) + last_available_file}"
        }])], ignore_index=True)

        # Save the metadata to a CSV file after each pair
        metadata_df.to_csv(f"{save_dir}/metadata.csv", index=False)

Crawling https://naslemana.com/2024/05/20/%d9%85%d8%a7%d9%87%d9%86%d8%a7%d9%85%d9%87-%d9%86%d8%b3%d9%84-%d9%85%d8%a7%d9%86%d8%a7-%d8%b3%d8%a7%d9%84-%d8%b3%d9%88%d9%85-%d8%b4%d9%85%d8%a7%d8%b1%d9%87-%db%b2%db%b9-%d8%a7%d8%b1/
